# 18-Oct-2022

## Summary of Goals
1. Computing the norm of the error in the update (error introduced by the SVD truncation and approximation of the DMD algorithm)
2. 

## 1. Uncertainty in our update
I think there are 3 error norms that we can compute:
1. Error of DMD approximation of the non-linear dynamics
    * This doesn't tell us about the error introduced by the SVD approximation
2. Computing the error by projecting the X and X' matrices onto X's POD modes
    * the error is smaller t
3. Computing the error by projecting Atilde back ionto the full-dimensional space
    * A little computationally expensive

In [ ]:
# A = X2@V@np.linalg.inv(S_matrix)@U.T
# diff = X2 - A@X1
# np.linalg.norm(diff, 'fro')

# diff = X2 - Ur@Atilde@Ur.T@X1
# np.linalg.norm(diff, 'fro')

# diff = Ur.T@X2 - Atilde@(Ur.T@X1)
# np.linalg.norm(diff, 'fro')